In [1]:
#Importar Librerias
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
import locale
locale.setlocale(locale.LC_ALL, 'en_US')
#Leer Archivo
base = pd.read_excel('13-winmeier.xlsx', sheet_name='Reportes de productividad', header=13)
#Eliminar columnas no deseadas
base = base.drop(base.columns[0], axis=1)
#Especificar columnas que voy a utilizar
columnas_especificas = ['Isla','Terminal','Jugado','Netwin']
base = base[columnas_especificas]
# Encontrar Total dentro de la columna Isla, para solo tomar en cuenta
total_index = base[base['Isla'] == 'TOTAL: '].index[0]
# Filter the DataFrame to include only rows before the "TOTAL:" row in the 'Isla' column
base = base.loc[:total_index - 1]
# Filtrar datos no deseados
base = base.loc[~base['Isla'].isin(['Subtotal:'])]
base = base.loc[~base['Terminal'].isin(['BONOPLAY'])]
base['Terminal'] = base['Terminal'].str.split('-', expand=True)[0]
print(base)

    Isla                  Terminal   Jugado  Netwin
1    ---       723003 / 6258600237   6044.5 -272.55
2    ---       723001 / 6258600235    818.5  398.27
3    ---       723002 / 6258600236     1811  -40.95
4    ---       723004 / 6258600238    679.5  239.64
6      1  006 / 632246 / XAW039691  1397.65 -180.14
..   ...                       ...      ...     ...
311  100     193 / 189108 / 104104   6038.8   719.6
312  100     191 / 189106 / 104102        0       0
313  100     192 / 189107 / 104103  5185.45    -395
314  100     195 / 189110 / 104106   1376.8  -307.9
315  100     198 / 189113 / 104109   5451.8  -14.75

[283 rows x 4 columns]


In [2]:
base['Terminal'] = base['Terminal'].str.split('/', expand=True)[1]
nuevos_nombres = {
        'Isla': 'isla',
        'Terminal': 'terminal',
        'Jugado': 'coin_in',
        'Netwin': 'net_win',
    }
base.rename(columns=nuevos_nombres, inplace=True)
# Nombre del archivo de salida
output_file = 'reporte_13.xlsx'

# Guardar el DataFrame modificado en un archivo Excel
base.to_excel(output_file, index=False)

# Imprimir el mensaje de confirmación
print(f'DataFrame guardado en {output_file}')
print(base)

DataFrame guardado en reporte_13.xlsx
    isla     terminal  coin_in net_win
1    ---   6258600237   6044.5 -272.55
2    ---   6258600235    818.5  398.27
3    ---   6258600236     1811  -40.95
4    ---   6258600238    679.5  239.64
6      1      632246   1397.65 -180.14
..   ...          ...      ...     ...
311  100      189108    6038.8   719.6
312  100      189106         0       0
313  100      189107   5185.45    -395
314  100      189110    1376.8  -307.9
315  100      189113    5451.8  -14.75

[283 rows x 4 columns]


In [3]:
apuestas = pd.read_excel('13-winmeier-apuestas.xlsx', sheet_name='Sheet1', header=6)
apuestas=apuestas.drop(apuestas.columns[0], axis=1)
columnas_necesarias=['Desde','Hasta','Nombre','Jugadas','Apuesta media']
apuestas=apuestas[columnas_necesarias]
total_index_1 = apuestas[apuestas['Desde'] == 'Total Mensual'].index[0]
apuestas = apuestas.loc[:total_index_1 -1]
apuestas = apuestas.loc[~apuestas['Hasta'].isin(['Subtotal:'])]
apuestas['Desde'] = pd.to_datetime(apuestas['Desde']).dt.strftime('%d/%m/%Y')
apuestas['Nombre'] = apuestas['Nombre'].str.split('-', expand=True)[0]
apuestas.insert(1, 'sala', '3')
apuestas.insert(2, 'fecha', '13/09/2023')
print(apuestas)

          Desde sala       fecha             Hasta                    Nombre  \
0    13/09/2023    3  13/09/2023  14/09/2023 07:00  077 / 860011 / XAR648382   
1    13/09/2023    3  13/09/2023  14/09/2023 07:00  080 / 860010 / XAR648379   
2    13/09/2023    3  13/09/2023  14/09/2023 07:00  081 / 860009 / XAR648349   
3    13/09/2023    3  13/09/2023  14/09/2023 07:00  082 / 860012 / XAR648489   
4    13/09/2023    3  13/09/2023  14/09/2023 07:00  319 / 860025 / XAR653941   
..          ...  ...         ...               ...                       ...   
278  13/09/2023    3  13/09/2023  14/09/2023 07:00   165 / 140095 / W2055054   
279  13/09/2023    3  13/09/2023  14/09/2023 07:00   166 / 140030 / W2023885   
280  13/09/2023    3  13/09/2023  14/09/2023 07:00   201 / 140077 / W2055055   
281  13/09/2023    3  13/09/2023  14/09/2023 07:00    290 / 160059 / 5015770   
282  13/09/2023    3  13/09/2023  14/09/2023 07:00    291 / 160053 / 5015763   

     Jugadas  Apuesta media  
0      72

C:\Users\Andrea Rosas\AppData\Local\Temp\ipykernel_15108\1348310958.py:8: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  apuestas['Desde'] = pd.to_datetime(apuestas['Desde']).dt.strftime('%d/%m/%Y')


In [4]:
apuestas['Nombre'] = apuestas['Nombre'].str.split('/', expand=True)[1]
apuestas.drop('Hasta', axis=1, inplace=True)
apuestas.drop('Desde', axis=1, inplace=True)
nuevos_nombres_1 = {
        'Nombre': 'terminal',
        'Jugadas': 'game_played',
        'Apuesta media': 'average_bet',
    }
apuestas.rename(columns=nuevos_nombres_1, inplace=True)
# Nombre del archivo de salida
output_file = 'apuesta_13.xlsx'

# Guardar el DataFrame modificado en un archivo Excel
apuestas.to_excel(output_file, index=False)
print(apuestas)

    sala       fecha  terminal  game_played  average_bet
0      3  13/09/2023   860011         726.0         0.25
1      3  13/09/2023   860010         848.0         0.54
2      3  13/09/2023   860009         733.0         0.47
3      3  13/09/2023   860012         635.0         0.46
4      3  13/09/2023   860025        3009.0         0.13
..   ...         ...       ...          ...          ...
278    3  13/09/2023   140095         794.0         0.34
279    3  13/09/2023   140030         903.0         0.43
280    3  13/09/2023   140077         240.0         0.62
281    3  13/09/2023   160059        1855.0         1.00
282    3  13/09/2023   160053        1911.0         0.47

[271 rows x 5 columns]
